In [3]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [4]:
path_json = "D:/SITEC/permisos/sitecsolucionesyservicios-67cee8fd8c65.json"          
scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive'] 

cred = service_account.Credentials.from_service_account_file(path_json, scopes = scopes)

drive = build('drive' , 'v3', credentials = cred)
sheets = build('sheets' , 'v4' , credentials = cred)

In [5]:
import os
from googleapiclient.http import MediaFileUpload


flat = sheets.spreadsheets()

def extractsheets(service,file_id,r):
    return flat.values().get(spreadhseetId = file_id,range = r).execute()

def hold_on_premises(file,path_to_save,save_name,system_file,):   #la ruta de almacenamiento tiene que venir con todo y nombre de archivo
    path = os.path.join(path_to_save,save_name)
    file.to_csv(path,sep = ',' , index = False, encoding = 'UTF-8')
    
    print(f'archivo guardado en la ruta {path_to_save}')


def hold_on_drive(file,save_name,drive_folder_id,):
    path = os.path.join('C:/Users/52473/AppData/Local/Temp', save_name)     #La ruta es de la carpeta temporal de cada sistema
    file.to_csv(path, sep = ',' , index = False, encoding = 'UTF-8')
    
    rawfile = MediaFileUpload(path, mimeType = 'text/CSV')
    metadata_file = { 'name': save_name,
                     'parents' : drive_folder_id}
    drive.files().create(body = metadata_file, media_body = rawfile, fields = 'id').execute()
    
    print(f"archivo con ID: {id} en la carpeta con ID: {drive_folder_id}")

In [116]:
#Extraemos todas los nombres de las cotizaciones y de sus ID´s por carpeta
import pandas as pd

def list_files(service):
    query = "mimeType = 'application/vnd.google-apps.folder'"
    res = service.files().list(q = query).execute()
    
    return (pd.DataFrame(res.get('files',[]))).iloc[:,2:4]

    
def filesname_folder(service, folder_id):
    query = f"'{folder_id}' in parents"
    res = service.files().list(q = query).execute()
    files = pd.DataFrame(res.get('files', []))
    return files['name'].tolist() if 'name' in files else []

def filesid_folder(service, folder_id):
    query = f"'{folder_id}' in parents"
    res = service.files().list(q = query).execute()
    files = pd.DataFrame(res.get('files', []))
    return files['id'].tolist() if 'id' in files else []


In [117]:
#utilizamos las funciones

#Creamos una tabla para los nombres de los archivos dentro de carpetas 

carpetas = list_files(drive)
h = carpetas['name'].values

files_dict = {nom_carpeta : filesname_folder(drive,folder_id) for nom_carpeta, folder_id in zip(carpetas['name'],carpetas['id'])}
    
#Rellenamos los espacios con nulos de ser necesario

max_files = max(len(files) for files in files_dict.values())
for key in files_dict:
    files_dict[key].extend([None]*(max_files-len(files_dict[key])))    

filesname_drive = pd.DataFrame(files_dict)  


#tabla con los Id´s de los archivos

carpetas = list_files(drive)
h = carpetas['name'].values

files_dict = {nom_carpeta : filesid_folder(drive,folder_id) for nom_carpeta, folder_id in zip(carpetas['name'],carpetas['id'])}
    
#Rellenamos los espacios con nulos de ser necesario

max_files = max(len(files) for files in files_dict.values())
for key in files_dict:
    files_dict[key].extend([None]*(max_files-len(files_dict[key])))    

filesid_drive = pd.DataFrame(files_dict)  


In [ ]:
#Extracción por lotes:

